In [1]:
import pandas as pd
df = pd.read_csv('/content/train.csv')

In [2]:
df = df[['class', 'tweet']]

In [3]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(df["tweet"].tolist(),
                                                                    df["class"].tolist(),
                                                                    test_size=0.2,
                                                                    random_state=42)

In [4]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text data
train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length', max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
import tensorflow as tf

def convert_to_tf_dataset(encodings, labels):
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    )).batch(16)  # Batch size 16

train_dataset = convert_to_tf_dataset(train_encodings, train_labels)
val_dataset = convert_to_tf_dataset(val_encodings, val_labels)

In [6]:
from transformers import TFBertForSequenceClassification

# Load BERT with 2 output classes (Hate/Non-Hate)
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
METRICS = ['accuracy', tf.keras.metrics.Precision(name = 'precision'), tf.keras.metrics.Recall(name = 'recall')]

In [8]:
num_classes = 3
train_dataset = train_dataset.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_dataset = val_dataset.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))

In [10]:
from transformers import create_optimizer
import tensorflow as tf
from tensorflow import keras


# Define training parameters
num_train_steps = len(train_dataset) * 3  # Total training steps (num_batches * epochs)

# Correct optimizer initialization
optimizer, schedule = create_optimizer(init_lr=5e-5, num_warmup_steps=0, num_train_steps=num_train_steps, weight_decay_rate=0.01)

# Compile the model
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=METRICS)

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=3)


Epoch 1/3
1240/1240 [==============================] - 105s 72ms/step - loss: 0.1715 - accuracy: 0.9374 - precision: 0.8596 - recall: 0.9650 - val_loss: 0.3057 - val_accuracy: 0.9024 - val_precision: 0.7826 - val_recall: 0.9463
Epoch 2/3
1240/1240 [==============================] - 86s 69ms/step - loss: 0.1203 - accuracy: 0.9572 - precision: 0.8813 - recall: 0.9846 - val_loss: 0.3223 - val_accuracy: 0.9034 - val_precision: 0.8117 - val_recall: 0.9455
Epoch 3/3
1240/1240 [==============================] - 86s 70ms/step - loss: 0.0710 - accuracy: 0.9765 - precision: 0.9207 - recall: 0.9921 - val_loss: 0.3637 - val_accuracy: 0.9016 - val_precision: 0.8507 - val_recall: 0.9371


In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
model.push_to_hub('hate_speech_classifier')

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]